<a href="https://colab.research.google.com/github/sudeepmenon17/PytorchforNN/blob/main/FT_for_VIV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-auth
from google.colab import auth
from google.colab import drive

drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
import numpy as np
from scipy.interpolate import griddata
from scipy.io import loadmat, savemat

# Load the .mat files
vor_data = loadmat('/content/drive/MyDrive/Re500_RV5_D0p9/VORTALL.mat')
time_data = loadmat('/content/drive/MyDrive/Re500_RV5_D0p9/Time.mat')
points_data = loadmat('/content/drive/MyDrive/Re500_RV5_D0p9/points.mat')
cccool_data = loadmat('/content/drive/MyDrive/Re500_RV5_D0p9/CCcool.mat')
# Extract necessary variables
VORTALL = vor_data['VORTALL']  # Vorticity data
Time = time_data['Time'].flatten()  # Time vector (flatten to 1D)
points = points_data['points']  # Points array
CC = cccool_data['CC']  # Colormap

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal.windows import hamming
from matplotlib.colors import ListedColormap
import os

# Load data
file_path = "VortZ_interp.mat"
data = loadmat(file_path)

Xnew = data['Xnew']  # Vorticity data
nxi = int(data['nxi'][0][0])  # Number of x grid points
nyi = int(data['nyi'][0][0])  # Number of y grid points
pxnew = data['pxnew']  # x-coordinates
pynew = data['pynew']  # y-coordinates
Time = data['Time'].flatten()  # Time array

# Load custom colormap
cccool_file = "CCcool.mat"
cccool_data = loadmat(cccool_file)['CC']
custom_cmap = ListedColormap(cccool_data)

# Parameters
f_values = [0.2 * i for i in range(1, 5)]  # Frequencies to analyze
It0 = 0  # Initial time index
Dt = 1  # Time step
nx = nxi  # Spatial dimensions
ny = nyi

# Time and frequency calculations
L = len(Time)
dt = Time[1] - Time[0]
Fs = 1 / (dt * 0.2)  # Sampling frequency
f = Fs * np.arange(0, L // 2 + 1) / L  # Frequencies

# Create directory for saving results
os.makedirs("Mode_Data", exist_ok=True)

def custom_dft(data, freq, dt, windowing=True):
    """
    Perform a custom Discrete Fourier Transform (DFT) on a specific frequency.
    """
    n = len(data)
    t = np.arange(n) * dt

    # Apply Hamming window
    if windowing:
        window = hamming(n)
        dataw = data * window
        k = 2  # Compensation factor
    else:
        dataw = data
        k = 1

    # Perform DFT
    temp = np.sum(dataw * np.exp(-2j * np.pi * freq * t))
    amp = k * np.abs(temp) * 2 / n  # Amplitude
    phase = np.angle(temp, deg=True)  # Phase in degrees
    return amp, phase

# Process each frequency
for freq_idx, f1 in enumerate(f_values):
    print(f"Processing frequency f1 = {f1}...")

    # Compute amplitudes for each grid point
    P2 = np.zeros((nx * ny, 1))
    pha = np.zeros((nx * ny, 1))

    for i in range(nx * ny):
        vorticity_data = Xnew[i, It0::Dt]  # Extract time series for grid point
        P2[i, 0], pha[i, 0] = custom_dft(vorticity_data, f1, dt, windowing=True)

    # Prepare mode for visualization
    Phi = P2[:, 0]
    vortmax = np.max(np.abs(Phi))
    VORTlims = [-vortmax, vortmax]

    # Reshape and plot mode
    Phi_reshaped = Phi.reshape((ny, nx))
    plt.figure(figsize=(10, 6))
    plt.imshow(Phi_reshaped, extent=[pxnew.min(), pxnew.max(), pynew.min(), pynew.max()],
               vmin=VORTlims[0], vmax=VORTlims[1], cmap=custom_cmap, origin='lower')
    plt.colorbar(label='Amplitude')
    plt.title(f'DFT Mode for f1 = {f1}')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

    # Save data
    output_file = f"Mode_Data/dft_St_{f1:.2f}.csv"
    np.savetxt(output_file, np.column_stack((pxnew, pynew, Phi)), delimiter=",",
               header="px,py,Phi", comments="")